In [1]:
!pip install torch



[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
!pip install nltk

     ---------------------------------------- 0.0/41.5 kB ? eta -:--:--
     ---------------------------------------- 0.0/41.5 kB ? eta -:--:--
     --------- ------------------------------ 10.2/41.5 kB ? eta -:--:--
     --------- ------------------------------ 10.2/41.5 kB ? eta -:--:--
     --------- ------------------------------ 10.2/41.5 kB ? eta -:--:--
     -------------------------------------  41.0/41.5 kB 196.9 kB/s eta 0:00:01
     -------------------------------------- 41.5/41.5 kB 166.5 kB/s eta 0:00:00
     ---------------------------------------- 0.0/57.7 kB ? eta -:--:--
     ------- -------------------------------- 10.2/57.7 kB ? eta -:--:--
     --------------------------------- ---- 51.2/57.7 kB 890.4 kB/s eta 0:00:01
     -------------------------------------- 57.7/57.7 kB 608.1 kB/s eta 0:00:00
   ---------------------------------------- 0.0/1.5 MB ? eta -:--:--
   --- ------------------------------------ 0.1/1.5 MB 4.0 MB/s eta 0:00:01
   ------- ----------------


[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


## Sentiment Analysis

In [2]:
import pandas as pd
import numpy as np

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split

from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Hp\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Hp\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\Hp\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

In [4]:
df=pd.read_csv('test.csv',encoding='latin1')

df=df[['text','sentiment']]

df.head()

,text,sentiment
0,Last session of the day http://twitpic.com/67ezh,neutral
1,Shanghai is also really exciting (precisely -...,positive
2,"Recession hit Veronique Branquinho, she has to...",negative
3,happy bday!,positive
4,http://twitpic.com/4w75p - I like it!!,positive


In [11]:
# Preprocessing the dataset

stop_words=set('./punkt_tab/english')

def preprocess(text):
    
    #step1: handle the non-string entries
    
    if not isinstance(text,str):
        text=''
    print(f"Original :{text}")
    
    #step2: Convert to lowercase
    
    text=text.lower()
    
    #step3: Tokenize
    
    tokens=word_tokenize(text)
    print(f"Tokens: {tokens}")
    
    #step4:Remove stopwords and non alphabet tokens
    
    clean_tokens=[]
    
    for w in tokens:
        if w.isalpha() and w not in stop_words:
            clean_tokens.append(w)
    print(f"Clean Tokens: {clean_tokens}")
    
    #step5: Join tokens back into the string
    
    clean_text=' '.join(clean_tokens)
    print(f"Final Clean Text: {clean_text}\n{'-'*40}")
    
    return clean_text

df['text']=df['text'].fillna('')

for i in range(5):
    preprocess(df.loc[i,'text'])
    
df['clean_text']=df['text'].apply(preprocess)

Original :last session of the day http
Tokens: ['last', 'session', 'of', 'the', 'day', 'http']
Clean Tokens: ['last', 'session', 'of', 'the', 'day', 'http']
Final Clean Text: last session of the day http
----------------------------------------
Original :shanghai is also really exciting precisely skyscrapers galore good tweeps in china sh bj
Tokens: ['shanghai', 'is', 'also', 'really', 'exciting', 'precisely', 'skyscrapers', 'galore', 'good', 'tweeps', 'in', 'china', 'sh', 'bj']
Clean Tokens: ['shanghai', 'is', 'also', 'really', 'exciting', 'precisely', 'skyscrapers', 'galore', 'good', 'tweeps', 'in', 'china', 'sh', 'bj']
Final Clean Text: shanghai is also really exciting precisely skyscrapers galore good tweeps in china sh bj
----------------------------------------
Original :recession hit veronique branquinho she has to quit her company such shame
Tokens: ['recession', 'hit', 'veronique', 'branquinho', 'she', 'has', 'to', 'quit', 'her', 'company', 'such', 'shame']
Clean Tokens: ['rec

In [12]:
def map_sentiment(sentiment):
    if sentiment=='positive':
        return 1
    else: 
        return 0
    
#Apply the function to create the 'label' column
    
df['label']=df['sentiment'].apply(map_sentiment)
    
df.head()

,text,sentiment,label,clean_text
0,last session of the day http,neutral,0,last session of the day http
1,shanghai is also really exciting precisely sky...,positive,1,shanghai is also really exciting precisely sky...
2,recession hit veronique branquinho she has to ...,negative,0,recession hit veronique branquinho she has to ...
3,happy bday,positive,1,happy bday
4,http like it,positive,1,http like it


In [14]:
#Feature Extraction by TF-IDF

from sklearn.feature_extraction.text import TfidfVectorizer

#Step 1:Create the TF-IDF Vectorizer

vectorizer=TfidfVectorizer(max_features=10)

#Step2:Fit and Transform

X_tfidf=vectorizer.fit_transform(df['clean_text'])

# Step 3: View the feature names (vocabulary)
print("Vocabulary / Features learned by the vectorizer:")
print(vectorizer.get_feature_names_out())
print("-" * 50)

# Step 4: Convert to dense array
X = X_tfidf.toarray()

# Step 5: Show the TF-IDF vectors for the first few rows
print("TF-IDF vectors for the first 3 texts:")
for i in range(1):
    print(f"Original Text: {df.loc[i, 'clean_text']}")
    print(f"TF-IDF Vector: {X[i]}")
    print("-" * 50)

# Step 6: Show shapes of final matrices
y = df['label'].values
print("Final shapes:")
print("TF-IDF feature matrix shape:", X.shape)
print("Labels shape:", y.shape)


Vocabulary / Features learned by the vectorizer:
['and' 'for' 'in' 'is' 'it' 'my' 'that' 'the' 'to' 'you']
--------------------------------------------------
TF-IDF vectors for the first 3 texts:
Original Text: last session of the day http
TF-IDF Vector: [0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
--------------------------------------------------
Final shapes:
TF-IDF feature matrix shape: (4815, 10)
Labels shape: (4815,)
